# Introduction

The given dataset contains information about various attributes of a gaming session. A gaimng session happens when a user plays a game on a device(identified by "ai5" field). A typical session starts with a "ggstart" event and ends with a "ggstop" event. There can be multiple "ggstart" and "ggstop" events in a particular session. 

In [1]:
#Lets do some imports
import collections, numpy
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

In [2]:
#variables for session and time calculations
#Variable for total sessions
total_sessions = 0
#Variable for valid sessions
valid_sessions= 0
# variable to count the total run time of valid sessions
total_validtime =0

## Parsing Data into python using JSON

In [3]:
# list to store the read lines
list =[]
import json
with open('ggevent.log') as f:
    #Reading lines from .log file.
    for line in f.readlines()[1:]:
        dict_of_line = json.loads(line)
        #Initiating a dictionary to store information from read lines
        data={}
        data["uid"] = str(dict_of_line["headers"]["ai5"])
        data["event"] =str(dict_of_line["post"]["event"])
        data["time"]=  datetime.strptime(dict_of_line["bottle"]["timestamp"], '%Y-%m-%d %H:%M:%S.%f')
        #appending dictionaries into a list
        list.append(data)  

In [5]:
# creating a dataframe using the list above
df = DataFrame(list)

In [6]:
#dataframe preview
df.head()

,event,time,uid
0,ggstart,2016-05-09 02:00:00.033775,335644267c1d5f04eaea7bc6f51b1861
1,ggstop,2016-05-09 02:00:00.036721,ca3071aad676bc963795a2b09635cdf0
2,ggstop,2016-05-09 02:00:00.041006,17412dec7d3d02c9b0b1c3d1c3571c5c
3,ggstart,2016-05-09 02:00:00.074181,f81167c854f1a0c86cab6188f9995824
4,ggstart,2016-05-09 02:00:00.084916,17412dec7d3d02c9b0b1c3d1c3571c5c


In [7]:
# creating a list of unique user ids
unique = df["uid"].unique()

## Creating a list of dataframes grouped by user ids

In [9]:
#creating a list of dataframes by user ids
data =[]
#for every unique user id in dataset a dataframe is stored in data list
for alpha in unique:
    data.append(df[df["uid"] ==alpha])
    

Since, data is susceptible to losses, avoiding those user ids that have only sinlge event in the dataset i.e. either ony "ggstart" or "ggstop". It is difficult to track run time for such sessions.

In [10]:
#keeping only those dataframes, having more than one event i.e. atleast one session 
data =[i for i in data if len(i)>1]

In [11]:
#checking the length of list
len(data)

15237

In [12]:
# A sample output from data
data[21]

,event,time,uid
31,ggstop,2016-05-09 02:00:00.946466,fdf56abdb1758fc523e52f4c9bb05803
46,ggstart,2016-05-09 02:00:01.583882,fdf56abdb1758fc523e52f4c9bb05803
3633,ggstop,2016-05-09 02:01:37.097355,fdf56abdb1758fc523e52f4c9bb05803
3667,ggstart,2016-05-09 02:01:37.852882,fdf56abdb1758fc523e52f4c9bb05803
7384,ggstop,2016-05-09 02:03:10.706513,fdf56abdb1758fc523e52f4c9bb05803
7419,ggstart,2016-05-09 02:03:11.550855,fdf56abdb1758fc523e52f4c9bb05803
10820,ggstart,2016-05-09 02:04:33.939888,fdf56abdb1758fc523e52f4c9bb05803
10871,ggstop,2016-05-09 02:04:35.203347,fdf56abdb1758fc523e52f4c9bb05803
16331,ggstop,2016-05-09 02:06:49.597796,fdf56abdb1758fc523e52f4c9bb05803
16378,ggstart,2016-05-09 02:06:50.754033,fdf56abdb1758fc523e52f4c9bb05803


Adding more columns to the dataframes. "Difference" column is meant to calculate time difference between "ggstart"  and "ggstop" events. Since, any event can occour first, to determine the run time of a session "ggstart" should preecede "ggstop" while calculating difference. Thus, "eve" column is used to assign 1 as value to "ggstart" event and 0 as value to "ggstop" event. The difference betwwen rows of "eve" corresponding to time difference will help to determine whether "ggstart" occoured first or "ggstop" or events repeated themseves.

In [15]:
#Function defind to create "eve" column
#Returns 1 if event is "ggstart" otherwise returns 0
def event_decider(doo):
    event = doo
    if event == "ggstart":
        return 0
    else:
        return 1

In [16]:
# Setting warning optin to None
pd.set_option('mode.chained_assignment',None)
#for every dataframe in data adding two more columns ,difference and eve
for row in data:
    #difference column is differnce of consecutive time cells.
    row["difference"] = (row["time"].diff(1)).dt.total_seconds().fillna(0)
    # eve column stores a value(1) if event is "ggstart" and (0) if event is "ggstop".
    row["eve"] = row["event"].apply(event_decider)

# Instructions 

# Solution function

From the data list, each dataframe calls event_decider function that calculates total sessions, valid sessions and total runtime for a particular user id. The sum of which gives overall value of these variables for the given dataset.
The flow of logic in function is as follows:

If eve_difference =1, "ggstart" comes before "ggstop", thus if the time difference corresponding to it is greater than 60 seconds, it will be counted as valid session and time will be added.

If eve_difference = -1, "ggstop" occurs first.The session is over if corresponding time difference>  30seconds. The time difference is not added to run time.

Eve_difference =0, reflects that there is some data loss in between.A session has been missed. For example, if two "ggstop" occurs consecutively,a "ggstart" event should be there in between them. I have assumed that the three events should be equally spaced in time, thus half of time difference is added to run-time and valid session count is also increased by 1.

In [17]:
# Function to calculate sessions and session time
def event_decider(frame):
    ts = 0
    vs = 0
    total_time = 0
    # using global variables defined above
    #Creating "eve_difference" column as difference of consecutive "eve" rows
    frame["eve_difference"] = (frame["eve"].diff(1)).fillna("Nan")
    #iterating each row of database
    for index, row in frame.iterrows():
        # applying logic- if successive event is "ggstop" and predecessive  event is "ggstart"
        if row["eve_difference"] ==1: 
            # checking if time difference b/w the two events if greater than 1 second
            if row["difference"]>1:
                ts +=1
                #if time difference is greater than 60 seconds, a valid session
                if row["difference"]> 60:
                    vs+=1
                    total_time +=row["difference"]
        # applying logic- if successive event is "ggstart" and predecessive  event is "ggstop"            
        if row["eve_difference"] ==-1:
            if row["difference"] >30:
                ts+=1
                vs+=1
        # applying logic- if consecutive events are same       
        if row["eve_difference"] ==0:
            if row["difference"]>1:
                if row["difference"]< 30:
                    ts+=1
                if row["difference"]>120:
                    ts+=1   
                    vs+=1
                    # Adding half of time difference, assuming one seesion occured b/w the two events.
                    total_time +=(row["difference"]/2)
            else:
                pass        
    return ts,vs,total_time     
  

In [18]:
# Iterating over each element of list and applying function to it to the return value of parametersfor each user id.
for row in data:
    (ts,vs,total_time) = event_decider(row)
    #Calculating final values of parameter for complete dataset
    total_sessions  += ts 
    valid_sessions  += vs
    total_validtime+=total_time
    

In [19]:
print valid_sessions,total_sessions,total_validtime

48783 80928 7311456.24877


# Reflections

The dataset contains information about game sessions. Out of the given parameters, "event", "timeStamp", "ts" and "ai5" were used to solve this problem. The data was susceptible to losses and thus, there were cases where an user id had only single event in the complete data set. Those user ids were not used in computation. Consecutive "ggstart" and "ggstop" events were tackled with an assumption that an event should be there in between them and the three being equally spaced in time should increase the run time by time_difference/2 and valid sessions count by 1.To scale the dataset by 100 times, SQL can be employed. The data can be stored in database and used to calculate the required parameters. 
 